# Numeros de SPAM

In [2]:
library(readr)
library(mxmaps)
library(dplyr)
library(RCurl)

columnas_log = c("accion", "pal_1", "fecha", "pal_2", "hora", "pal_3", "tipo_linea", "pal_4", "numero", "pal_5", "pal_6", "pal_7", "nombre", "meh")

## Readr imprime "Parsed with column specification", con esto se desactiva
options(readr.num_columns = 0)

ladas_mexico <- read_csv("/home/pi/notebooks/Python/datasets/ladas_mexico.csv")

## Descargar archivo en binario
bin_file <- getBinaryURL(url = "sftp://pi@192.168.1.152/home/pi/log_llamadas.log", userpwd="pi:miPassword")

## Agregarle datos al archivo existente
fconn <- file("/home/pi/notebooks/Python/datasets/log_llamadas.log", "ab") # ab = append+binary

## Escribir el archivo bajado en el archivo existente
writeBin(bin_file, fconn)

## Cerrar archivo.
close(fconn)

## Cargar archivo que recién descargamos
numeros_bloqueados <- read_delim("/home/pi/notebooks/Python/datasets/log_llamadas.log", delim="*", col_names = columnas_log)

# Eliminar datos duplicados
numeros_bloqueados <-
    numeros_bloqueados %>%
        distinct(fecha, hora, numero)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: bitops


In [11]:
numeros_bloqueados <- numeros_bloqueados[,c("fecha", "hora", "numero")]
numeros_bloqueados$lada=substr(numeros_bloqueados$numero, 1, 3)

ladas_mexico <-
    ladas_mexico %>%
    group_by(lada, estado) %>%
    count() %>%
    select(lada, estado)

# La lada 56 también es de la CDMX, pero no está en el listado porque es nueva

datos_completos <- merge(numeros_bloqueados, ladas_mexico, by="lada", all.x=TRUE)

llamadas_x_estado <-
    datos_completos %>%
    mutate(lada = ifelse(is.na(estado), substr(lada, 1,2), lada)) %>%
    mutate(lada = ifelse(lada=="56", "55", lada)) %>%
    mutate(lada = as.numeric(lada)) %>%    
    left_join(ladas_mexico, by="lada") %>% 
    mutate(estado.y = ifelse(estado.y=="Distrito Federal", "Ciudad de México", estado.y)) %>%
    mutate(estado.y = ifelse(estado.y=="Estado de México", "México", estado.y)) %>%    
    select(numero, estado.y)

conteo_x_estados <- 
    llamadas_x_estado %>%
    group_by(estado.y) %>%
    count()

names(conteo_x_estados)<-c("estado", "value")

data("df_mxstate")

datos_mexico <- df_mxstate

datos_mexico <-
    datos_mexico %>%
    select(region, state_name) %>%
    left_join(conteo_x_estados, by=c("state_name"="estado")) %>%
    mutate(value = ifelse(is.na(value), 0, value))

mapita_plot <- mxstate_choropleth(datos_mexico, num_colors = 1, title = "Llamadas de cobranza")

# Para graficar el mapa con Plotly:
library(plotly)
library(IRdisplay)


interactive_plot <- ggplotly(mapita_plot)
# en Chrome sólo se imprime el plot y ya.
# interactive_plot

# Para que funcione en Firefox, se tiene que embeber en un iframe.
htmlwidgets::saveWidget(interactive_plot, "grafiquita.htm", selfcontained = FALSE)
display_html('<iframe width=900 height=650 src="grafiquita.htm"></iframe>')